In [1]:
from impresso import connect

impresso = connect()

🎉 You are now connected to the Impresso API!  🎉
🔗 Using API: http://localhost:3030


# Named entity recognition


In [2]:
text = """
Jean-Baptiste Nicolas Robert Schuman ( 
29 June 1886 – 4 September 1963) was a Luxembourg-born French 
statesman. Schuman was a Christian democratic (Popular 
Republican Movement) political thinker and activist. 
"""
result = impresso.tools.ner(
    text=text
)
result

,type,surfaceForm,function,name,confidence.ner,offset.start,offset.end,wikidata.id,wikidata.wikipediaPageName
id,,,,,,,,,
1:37:pers:ner-stacked-2-bert-medium-historic-multilingual|ner-mgenre-multilingual,pers,Jean-Baptiste Nicolas Robert Schuman,N/A,Baptiste Nicolas Robert Schuman,91.25,1,37,N/A,N/A
41:72:time:ner-stacked-2-bert-medium-historic-multilingual|ner-mgenre-multilingual,time,29 June 1886 – 4 September 1963,N/A,N/A,77.90,41,72,N/A,N/A
80:90:org:ner-stacked-2-bert-medium-historic-multilingual|ner-mgenre-multilingual,org,Luxembourg,N/A,N/A,25.12,80,90,N/A,N/A


In [3]:
text = """
Jean-Baptiste Nicolas Robert Schuman ( 
29 June 1886 – 4 September 1963) was a Luxembourg-born French 
statesman. Schuman was a Christian democratic (Popular 
Republican Movement) political thinker and activist. 
"""
result = impresso.tools.ner_nel(
    text=text,
)
result

,type,surfaceForm,function,name,confidence.ner,confidence.nel,offset.start,offset.end,wikidata.id,wikidata.wikipediaPageName,wikidata.wikipediaPageUrl
id,,,,,,,,,,,
1:37:pers:nel-mgenre-multilingual,pers,Jean-Baptiste Nicolas Robert Schuman,N/A,Baptiste Nicolas Robert Schuman,91.25,96.76,1,37,Q15981,Robert Schuman,https://en.wikipedia.org/wiki/Robert_Schuman
41:72:time:nel-mgenre-multilingual,time,29 June 1886 – 4 September 1963,N/A,N/A,77.90,86.46,41,72,Q15981,Robert Schuman,https://en.wikipedia.org/wiki/Robert_Schuman
80:90:org:nel-mgenre-multilingual,org,Luxembourg,N/A,N/A,25.12,100.00,80,90,Q32,Luxembourg,https://en.wikipedia.org/wiki/Luxembourg


In [4]:
text = """
[START]Jean-Baptiste Nicolas Robert Schuman[END] ( 
29 June 1886 – 4 September 1963) was a Luxembourg-born French 
statesman. Schuman was a Christian democratic (Popular 
Republican Movement) political thinker and activist. 
"""
result = impresso.tools.nel(
    text=text,
)
result

,type,confidence.nel,wikidata.id,wikidata.wikipediaPageName,wikidata.wikipediaPageUrl
id,,,,,
,unk,99.94,Q15981,Robert Schuman,https://en.wikipedia.org/wiki/Robert_Schuman


# Embeddings

In [23]:
embedding = impresso.tools.embed_text(text="hello world", target="text")
embedding

'gte-768:jK0GvVuqST1ZVeu8UOozOxDawLs42s48R715vYjiiD1T/SQ9W5CIvQNVzLx8EG6724cHvtNcbT3qSWO9vPedPQFZAz61qCI9JfA4Pb9MqDwQ49E8ANO6OzyWHL00B4A895rLPF/9hz1c2rs9/w9Avax6Nr0PE7g6QSisvaMMlr2buFY8rW6iu204pz05bDa9JNmkvJlLPD2YtWM8HvWEvQB65Lxp+Ga9n0mBvA+EeT1k2jq8woA2PGkZzLt466A9UyMlPU9fULsPBIw8MJ/Ku0Ox/TyWiY273TUZPdXGdbt29Eq9b4aaPOnwgT36W8s7BtwWPXeiPbuKT1y8SsBtvE2LTL1J+5s7k4DCOfAqUbyRgi4+L0shugB1G73RF9E8krcMvd+PQbzIuiK9xLUbPeB7t71G8Di87R2svNODAD3TWVO87FdLPBC0pjzoXhg9PR3ovEjeVL3tjo08/K5bvA5DMbxUaFy6THuNPRp+aDwzQoA95vqbvAliebtpBCm8H3l2vJaXIrzd1Ew9DqEvPAxMULy33qA73zm1O6SKDr0iV4q8DOxNOkF03zwDEhi7SoNXvdngd7zhHqK7z7FJvMNUjb01kxS8LW0aOaslyDyyiRc9QO1CPEO6dD0tSWU8RApivM+4hz2weRi8cLWmPPCG6r0+Uxc8fIcUPFOrNDy+FJg8oKmHvA8HUj3TY4s9J5xGvR7ffb3Nfpg5Z+cRvRQEDzzhlv49HrMCvJjOEjxfV6e9HWGsPUdprL2na987l6cyPQLCHDzHB429EZH0vLBSezvnHd48wyUivQ9pujybNJ49mS3CvHa/hjzyyPO8G/0HvR0YkzusZfW8yFxJvAjokj3v4QY8ewnzO1NCXL0VR4C93u83uzqzCLwKzIY8Szr8PAyrszwdlH69ERvVOxeOuzxZepG70GaIvMH05DztlJW85nP3PGU7aL0Gjrs8G00YPRXM07yvWKi8NgCKvOERLT1vUsK

In [45]:
impresso.search.find(
  embedding=impresso.tools.embed_text(text="Schumann polititian", target="text"),
  limit=3
)

,copyrightStatus,type,sourceMedium,title,locationEntities,personEntities,organisationEntities,newsAgenciesEntities,topics,transcriptLength,totalPages,languageCode,isOnFrontPage,publicationDate,issueUid,countryCode,providerCode,mediaUid,mediaType
uid,,,,,,,,,,,,,,,,,,,
JDG-1963-09-06-a-i0004,in_cpy,ar,print,"Robert Schuman, prince de l'Europe","[{'uid': '2-54-Europe', 'count': 2}]","[{'uid': '2-50-Robert_Schuman', 'count': 5}]",[],[],"[{'uid': 'tm-fr-all-v2.0_tp55_fr', 'relevance'...",390,1,fr,True,1963-09-06T00:00:00+00:00,JDG-1963-09-06-a,CH,SNL,JDG,newspaper
luxland-1963-09-06-a-i0046,in_cpy,ar,print,Mon ami Robert Schuman,"[{'uid': '2-54-Luxembourg', 'count': 1}, {'uid...","[{'uid': '2-50-Robert_Schuman', 'count': 3}]",[],[],"[{'uid': 'tm-fr-all-v2.0_tp99_fr', 'relevance'...",364,1,fr,False,1963-09-06T00:00:00+00:00,luxland-1963-09-06-a,LU,BNL,luxland,newspaper
IMP-1950-05-19-a-i0001,in_cpy,ar,print,Le «combinat» franco-allemand charbon et acier,[],[],[],[],"[{'uid': 'tm-fr-all-v2.0_tp87_fr', 'relevance'...",443,1,fr,True,1950-05-19T00:00:00+00:00,IMP-1950-05-19-a,CH,SNL,IMP,newspaper
